In [2]:
import soundcloud
import pandas as pd
import time
from ast import literal_eval
from collections import MutableMapping

from bs4 import BeautifulSoup
import requests
import urllib
import csv
import numpy as np
import psycopg2

In [2]:
c_id = '887b335a80f3e625454ebca548c53d96'
client = soundcloud.Client(client_id=c_id)

In [3]:
df = pd.read_csv('../data/sc_aliases.csv')

In [6]:
df = df[(df['sc_alias']!= 'x') & (df['sc_alias'] != 'No Soundcloud Found')]

In [57]:
artists = df['artist_name'].values

In [54]:
aliases = df['sc_alias'].values

In [58]:
artist_alias = zip(artists, aliases)

## Connect to db

In [44]:
conn = psycopg2.connect(dbname='soundcloud', user='postgres')

In [45]:
c = conn.cursor()

## Create a table

In [51]:
query = """
CREATE TABLE sc_artists (
    id SERIAL PRIMARY KEY
    ,artist_name VARCHAR
    ,sc_alias VARCHAR
    ,city VARCHAR
    ,country VARCHAR
    ,followers_count VARCHAR
    ,followings_count VARCHAR
    ,last_modified VARCHAR
    ,playlist_count VARCHAR
    ,plan VARCHAR
    ,public_favorites_count VARCHAR
    ,track_count VARCHAR
);
"""

In [52]:
c.execute(query)

In [53]:
conn.commit()

## Insert rows into table

In [56]:
query = """
INSERT INTO
    sc_artists
    (artist_name, sc_alias, city, country, followers_count, followings_count, last_modified, playlist_count, plan, public_favorites_count, track_count)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [71]:
def get_artist_info(artist_alias):
    total = str(len(artist_alias))
    for index, artist_alias in enumerate(artist_alias):
        try:
            artist = artist_alias[0]
            alias = artist_alias[1]
            url = 'https://soundcloud.com/' + alias
            artist_info = client.get('/resolve', url=url, linked_partitioning=1, limit=200).fields()
            c.execute(query, (str(artist), str(alias), artist_info['city'], artist_info['country'], str(artist_info['followers_count']), str(artist_info['followings_count']), str(artist_info['last_modified']), str(artist_info['playlist_count']), artist_info['plan'], str(artist_info['public_favorites_count']), str(artist_info['track_count']))) 
            conn.commit()
            if index % 25 == 0:
                print 'Artist {} of {} complete ({})'.format(str(index), total, artist)
        except Exception as e:
            conn.commit()
            print 'Artist {} of {} failed ({}) due to exception: {}'.format(str(index), total, artist, str(e))

In [154]:
foreign_artists = [('Hernán Cattáneo', 'hernancattaneo')
,('Tiësto', 'tiesto')
,('Sven Väth','sven-vath-is-50')
,('Trentemøller', 'trentemusic')
,('Andrés', 'roman-fluegel')
,('András', 'andrasfox')
,('Roman Flügel','roman-fluegel')
,('Kornél Kovács','kornel')
,('Lindstrøm','feedelity')
,('D•K•','d-k-music')
,('Âme', 'ame')
,('Ц5 AҪP', 'officialc5')
,("NHK'Koyxeи", 'nhk_koyxen')
,('Dürerstuben', 'duererstuben')
,('Miss Roisin Murphy','miss-roisin-murphy')
,('Kickflip Mike','joschka-seibt')
,('Ricoshëi','ricoshei')
,('Davíð Þór','say-dave')
,('RÜFÜS','rufussounds')
,('Bjørn Torske','bjorn-torske')
,('Luciano','lucianocadenza')
,('RUFDUG','rufdug')
,('Der Ausländer','der_auslander')
,('Kasper Bjørke','kasperbjorke')
,('Chancha Via Circuito', 'chanchaviacircuito')
,('The Knife','theknife')
,('arrrgh°','arrrrrrrgh')
,('Ñaka Ñaka','naka-naka')
,('Marko Fürstenberg','markofuerstenberg')
,('christopher schwarzwalder','shuffle')
,('Jesper Dahlbäck','jesperdahlback')
,('Øyvind Morken','oyvind-morken')
,('Felipe Sá','felipe-sa')
,('gebrueder teichmann','gebrueder-teichmann')
,('Pascäal','pascaaal')
,('René Najera','rene-najera')
,('Stefan Jós','stefan_jos')
,('Matias Aguayo','donmatiasaguayo')
,('Alli Borèm','alli-bor-m')
,('Tim Vitá','timvita')
,('Empfänger','empfanger')
,('Infrasonicrec','infrasonicrec')
,('Niccolò Cupo','niccolocupo')
,('Bogdan Dražić','bogdan-drazic')
,('Raär','11pmraar')                  
,('T. Page','tpage001')
,('Jäfar','jfar')
,('Rodion G.A.','rodionga')
,('Prins Thomas','prinsthomas')
,('Diego Gamez','gamezdiego')
,('Ivan Dorn','ivandorn-com')
,('Javi Frias','javi-frias')
,('Dan Noël','dannoel')                 
,('Sísý Ey','sisyey')                   
,('CASHBY Δ','cashby')                   
,('ultraísta','ultra1sta')                   
,('Lutzenkirchen','lutzenkirchen')                   
,('Motëm','motem')                   
,('Kool D','musclerecords')
,('Raíz','raiz-droid')
,('Fredrik Stjärne','fredrikstjarne')                  
,('Lutzenkirchen','lutzenkirchen')                  
,('Soren','sorenmakesmusic')                  
,('Dejan Milicevic','dj-dejan-milicevic')                  
,('Sven von Thülen','svt')
,('Thorsteinssøn','thorsteinsson')
,('マクロスMACROSS 82-99','macross-82-99')
,("Mr. K 'Alexi",'mr-k-alexi')                   
,('Mathias Schaffhäuser','mathias-schaffhauser')                   
]

In [169]:
get_artist_info(foreign_artists[2:])

Artist 0 of 67 complete (Sven Väth)
Artist 25 of 67 complete (Ñaka Ñaka)
Artist 50 of 67 complete (Dan Noël)


In [3]:
df = pd.read_csv('../data/sc_artists.csv')

In [4]:
df_dups = pd.read_csv('../data/duplicate_artists.csv')

In [6]:
df_withdups = df.merge(df_dups, how='left', on='sc_alias')

In [22]:
df_withdups[pd.notnull(df_withdups['count'])].sort(['sc_alias']).to_csv('../data/dup_artists_list.csv')

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [74]:
get_artist_info(artist_alias)

Artist 0 of 7222 complete (49ers)
Artist 25 of 7222 complete (The Aston Shuffle)
Artist 50 of 7222 complete (Brother Brown)
Artist 75 of 7222 complete (Curtis Jones)
Artist 84 of 7222 failed (Daphne Rubin-Vega) due to exception: 404 Client Error: Not Found for url: https://api.soundcloud.com/resolve?url=https%3A%2F%2Fsoundcloud.com%2Fupload&linked_partitioning=1&limit=200&client_id=887b335a80f3e625454ebca548c53d96
Artist 94 of 7222 failed (Deskee) due to exception: 404 Client Error: Not Found for url: https://api.soundcloud.com/resolve?url=https%3A%2F%2Fsoundcloud.com%2Fupload&linked_partitioning=1&limit=200&client_id=887b335a80f3e625454ebca548c53d96
Artist 100 of 7222 complete (Dirty Vegas)
Artist 125 of 7222 complete (Eddie Amador)
Artist 150 of 7222 complete (Global Deejays)
Artist 162 of 7222 failed (Hern��n Catt��neo) due to exception: invalid byte sequence for encoding "UTF8": 0xcc 0xc1

Artist 175 of 7222 complete (James Holden)
Artist 200 of 7222 complete (Kavinsky)
Artist 225 

In [29]:
df_dups_cleaned = pd.read_csv('../data/dup_artists_list.csv')

In [35]:
df_withdups = df_withdups[df_withdups['count'].isnull()]

In [32]:
df_dups_cleaned = df_dups_cleaned[df_dups_cleaned['Unnamed: 2']=='x']

In [33]:
df_dups_cleaned.drop(['Unnamed: 0', 'Unnamed: 2'], inplace=True, axis=1)

In [54]:
df_dups_cleaned[df_dups_cleaned['followers_count']<500]

,id,artist_name,sc_alias,city,country,followers_count,followings_count,last_modified,playlist_count,plan,public_favorites_count,track_count,count
84,4373,K.K,bigkk100,Oxon Hill,NaN,99,0,2016/07/18 02:01:04 +0000,0,Free,25,10,2
85,699,Black Ice Productions,black-ice,NaN,NaN,3,31,2015/08/09 08:54:24 +0000,5,Free,23,0,2
86,950,Black Ice,black-ice,NaN,NaN,3,31,2015/08/09 08:54:24 +0000,5,Free,22,0,2
184,6622,Bowie,davidbowiewemissyou,Perth,Australia,235,134,2016/06/17 09:57:01 +0000,0,Free,46,35,2
185,6356,WA,davidbowiewemissyou,Perth,Australia,235,134,2016/06/17 09:57:01 +0000,0,Free,46,35,2
187,3590,D. B.,dbtharasta,"St. Paul, MN",United States,495,783,2016/11/21 07:07:03 +0000,2,Free,0,51,2
305,1403,Admas,egapmada,Adelaide,Australia,248,62,2016/10/16 09:28:55 +0000,3,Pro Plus,3,28,2
306,3840,Adam,egapmada,Adelaide,Australia,249,62,2016/10/16 09:28:55 +0000,3,Pro Plus,3,28,2
339,5992,Federico Lijtmaer,fedelijt,NaN,NaN,458,54,2016/04/21 18:07:38 +0000,1,Free,4,2,2
375,2928,Diego Gamez,gamezdiego,Barcelona,Spain,292,88,2016/11/04 17:48:52 +0000,0,Pro Plus,12,5,2


In [36]:
df_artists = pd.concat([df_dups_cleaned, df_withdups])

In [38]:
len(df_artists)

6691

In [43]:
df_artists['plan'].unique()

array(['Pro', 'Pro Plus', 'Free'], dtype=object)

In [47]:
df_artists['followers_count'][0].values[0]

25323

In [48]:
df_artists = df_artists[(df_artists['plan']=='Pro') | (df_artists['plan']=='Pro Plus') | (df_artists['followers_count']>=500)]

In [52]:
df_artists.to_csv('../data/artist_meta.csv')

In [ ]:
for alias in aliases[0:5]:
    url = 'https://soundcloud.com/' + alias + '/followers'
    try:
        followers = client.get('/resolve', url=url, linked_partitioning=1, limit=page_size)
        fols = [fol['username'] for fol in followers.obj['collection']]
        url = followers.obj.get('next_href')
        while url is not None:
            followers = client.get(url.split('api.soundcloud.com')[1], linked_partitioning=1, limit=page_size)
            fols.extend([fol['username'] for fol in followers.obj['collection']])
            url = followers.obj.get('next_href')
        all_followers.extend(zip(([alias for fol in xrange(len(fols))], fols)))
    except:
        continue

In [ ]:
for alias in aliases[0:5]:
    url = 'https://soundcloud.com/' + alias + '/followers'
    try:
        followers = client.get('/resolve', url=url, linked_partitioning=1, limit=page_size)
        for fol in followers.obj['collection']:
            folls = folls.append({'alias': alias, 'follower': fol['username']}, ignore_index=True)
        url = followers.obj.get('next_href')
        while url is not None:
            followers = client.get(url.split('api.soundcloud.com')[1], linked_partitioning=1, limit=page_size)
            for fol in followers.obj['collection']:
                folls = folls.append({'alias': alias, 'follower': fol}, ignore_index=True)
                url = followers.obj.get('next_href')
        print alias + ' worked'
    except Exception, e:
        print alias + ' did not work', str(e)
        continue